In [0]:
import pandas as pd
import numpy as np

In [0]:
data = pd.read_csv('Data_for_preprocessing.csv')
data.head(2)

,review_text,aspects
0,CaliBamboo is a premium bamboo flooring produc...,flooring:neutral
1,How can that happen? It happened because quart...,countertop:negative


In [0]:
data[['aspect1','aspect2','aspect3','aspect4','aspect5']] = data.aspects.str.split(';',expand = True)
data.head()

,review_text,aspects,aspect1,aspect2,aspect3,aspect4,aspect5
0,CaliBamboo is a premium bamboo flooring produc...,flooring:neutral,flooring:neutral,None,None,None,None
1,How can that happen? It happened because quart...,countertop:negative,countertop:negative,None,None,None,None
2,I love this and did something like it in my ho...,space_usage:positive,space_usage:positive,None,None,None,None
3,I think it will work. I painted my small bedro...,color:positive; space:positive,color:positive,space:positive,None,None,None
4,I think you need to change the kick plate to b...,molding:positive; doors:negative,molding:positive,doors:negative,None,None,None


In [0]:
data[['aspect11','sentiment1']] = data.aspect1.str.split(':',expand=True)
data[['aspect21','sentiment2']] = data.aspect2.str.split(':',expand=True)
data[['aspect31','sentiment3']] = data.aspect3.str.split(':',expand=True)
data[['aspect41','sentiment4']] = data.aspect4.str.split(':',expand=True)
data[['aspect51','sentiment5']] = data.aspect5.str.split(':',expand=True)
data.drop(['aspect1','aspect2','aspect3','aspect4','aspect5'],axis = 1,inplace=True)


In [0]:
data.head(2)

,review_text,aspects,aspect11,sentiment1,aspect21,sentiment2,aspect31,sentiment3,aspect41,sentiment4,aspect51,sentiment5
0,CaliBamboo is a premium bamboo flooring produc...,flooring:neutral,flooring,neutral,None,None,None,None,None,None,None,None
1,How can that happen? It happened because quart...,countertop:negative,countertop,negative,None,None,None,None,None,None,None,None


In [0]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
nltk.download('stopwords')

#Spacy
import spacy
nlp = spacy.load('en')

# Other
import re
import json
import string
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

#Keras
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, Activation

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Using TensorFlow backend.


In [0]:
absa_model = Sequential()
absa_model.add(Dense(512, input_shape=(100,), activation='relu'))
absa_model.add((Dense(256, activation='relu')))
absa_model.add((Dense(128, activation='relu')))
absa_model.add(Dense(78, activation='softmax'))
#compile model
absa_model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [0]:
from keras.preprocessing.text import Tokenizer
vocab_size = 100 # We set a maximum size for the vocabulary
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(data.review_text)
reviews_tokenized = pd.DataFrame(tokenizer.texts_to_matrix(data.review_text))

In [0]:
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
label_encoder1 = LabelEncoder()
integer_category = label_encoder1.fit_transform(data.aspect11)
encoded_y = to_categorical(integer_category)




In [0]:
#fit aspect classifier
absa_model.fit(reviews_tokenized,encoded_y, epochs=100, verbose=1,batch_size=100)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/100
465/465 [==============================] - 3s 7ms/step - loss: 4.2785 - acc: 0.0731
Epoch 2/100
465/465 [==============================] - 0s 86us/step - loss: 3.9465 - acc: 0.1097
Epoch 3/100
465/465 [==============================] - 0s 66us/step - loss: 3.6244 - acc: 0.1183
Epoch 4/100
465/465 [==============================] - 0s 65us/step - loss: 3.4418 - acc: 0.2086
Epoch 5/100
465/465 [==============================] - 0s 69us/step - loss: 3.2730 - acc: 0.2495
Epoch 6/100
465/465 [==============================] - 0s 65us/step - loss: 3.1031 - acc: 0.2688
Epoch 7/100
465/465 [==============================] - 0s 70us/step - loss: 2.9311 - acc: 0.3204
Epoch 8/100
465/465 [==============================] - 0s 64us/step - loss: 2.7452 - acc: 0.3699
Epoch 9/100
465/465 [==============================] - 0s 69us/step - loss: 2.5744 - acc: 0.3957
Epoch 10/100
465/465 [===========

In [0]:
# For Sentiment 
data.sentiment1.replace('negarive','negative',inplace=True)
data.sentiment1.replace(' neutral','neutral',inplace=True)
data.sentiment1.replace('negative\n','negative',inplace=True)
data.sentiment1.replace('positive\n','positive',inplace=True)

In [0]:
#model architecture
sentiment_model = Sequential()
sentiment_model.add(Dense(512, input_shape=(100,), activation='relu'))
sentiment_model.add((Dense(256, activation='relu')))
sentiment_model.add((Dense(128, activation='relu')))
sentiment_model.add(Dense(3, activation='softmax'))
#compile model
sentiment_model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

#encode the label variable
label_encoder = LabelEncoder()
integer_sentiment = label_encoder.fit_transform(data.sentiment1)
encoded_y = to_categorical(integer_sentiment)

In [0]:
# #fit sentiment classifier
sentiment_model.fit(reviews_tokenized, encoded_y ,epochs=100, verbose=1,batch_size=100)

Epoch 1/100
465/465 [==============================] - 0s 704us/step - loss: 0.9556 - acc: 0.5054
Epoch 2/100
465/465 [==============================] - 0s 69us/step - loss: 0.8481 - acc: 0.7054
Epoch 3/100
465/465 [==============================] - 0s 61us/step - loss: 0.7336 - acc: 0.7656
Epoch 4/100
465/465 [==============================] - 0s 69us/step - loss: 0.6333 - acc: 0.7828
Epoch 5/100
465/465 [==============================] - 0s 63us/step - loss: 0.5319 - acc: 0.8086
Epoch 6/100
465/465 [==============================] - 0s 64us/step - loss: 0.4425 - acc: 0.8323
Epoch 7/100
465/465 [==============================] - 0s 63us/step - loss: 0.3630 - acc: 0.8473
Epoch 8/100
465/465 [==============================] - 0s 61us/step - loss: 0.2817 - acc: 0.8946
Epoch 9/100
465/465 [==============================] - 0s 69us/step - loss: 0.2117 - acc: 0.9333
Epoch 10/100
465/465 [==============================] - 0s 80us/step - loss: 0.1517 - acc: 0.9570
Epoch 11/100
465/465 [======

In [0]:
test_reviews = [
'Oh what a beautiful brick home you have!','The roof looks fine.  I love your door color.  Add more color plants in front of the porch.',
'the space was not good','Design combination of triangular roof with a curved space below is beautiful!',
 'I\'m a fire guy as well. However, in an island, I think it\'s going to depend on your venting. A surface-mount downdraft sucks a lot of the BTUs from a gas flameâ€”that\'s what we had it in our last house. With proper overhead venting, that\'s not a concern.'
]
# Aspect preprocessing
test_reviews = [review.lower() for review in test_reviews]
test_aspect_terms = []
for review in nlp.pipe(test_reviews):
    chunks = [(chunk.root.text) for chunk in review.noun_chunks if chunk.root.pos_ == 'NOUN']
    test_aspect_terms.append(' '.join(chunks))
print(test_aspect_terms)

test_aspect_terms = pd.DataFrame(tokenizer.texts_to_matrix(test_aspect_terms))
print(test_aspect_terms)

# Sentiment preprocessing
test_sentiment_terms = []
for review in nlp.pipe(test_reviews):
        if review.is_parsed:
            test_sentiment_terms.append(' '.join([token.lemma_ for token in review if (not token.is_stop and not token.is_punct and (token.pos_ == "ADJ" or token.pos_ == "VERB"))]))
        else:
            test_sentiment_terms.append('') 
test_sentiment_terms = pd.DataFrame(tokenizer.texts_to_matrix(test_sentiment_terms))



['home', 'roof color plants front porch', 'space', 'combination roof space', 'guy island venting downdraft lot btus gas house venting concern']
    0    1    2    3    4    5    6   ...   93   94   95   96   97   98   99
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
3  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
4  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[5 rows x 100 columns]


In [0]:
# Models output
test_aspect_categories =  label_encoder1.inverse_transform(absa_model.predict_classes(test_aspect_terms))
print(test_aspect_categories)
test_sentiment = label_encoder.inverse_transform(sentiment_model.predict_classes(test_sentiment_terms))


['design' 'color' 'design' 'design' 'windows']


In [0]:
for i in range(len(test_sentiment)):
    print("Review " + str(i+1) + " is expressing a  " + test_sentiment[i] + " : " + test_aspect_categories[i])

Review 1 is expressing a  positive : design
Review 2 is expressing a  positive : color
Review 3 is expressing a  positive : design
Review 4 is expressing a  positive : design
Review 5 is expressing a  negative : windows
